In [1]:
import os
from dataclasses import dataclass
from pathlib import Path

In [2]:
%pwd

'd:\\Data Science\\Text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Text-summarizer'

In [5]:
#entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path

In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml_config,create_directorys

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directorys([self.config.artifacts_root]) # config hold a ConfigBox type so we can access its attribites by just config.attribute
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directorys([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config


     

In [8]:
import os
import zipfile
import urllib.request as request
from textSummarizer.utils.common import  get_size
from textSummarizer.logging import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
            logger.info(f"Downloaded {filename} - wiht following headers: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2024-08-16 13:41:28,722: INFO: common: Successfully loaded configuration from config\config.yaml]
[2024-08-16 13:41:28,724: INFO: common: Successfully loaded configuration from params.yaml]
[2024-08-16 13:41:28,726: INFO: common: Created directory: artifacts]
[2024-08-16 13:41:28,727: INFO: common: Created directory: artifacts/data_ingestion]
[2024-08-16 13:42:55,402: INFO: 85877278: Downloaded artifacts/data_ingestion/data.zip - wiht following headers: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E60A:320EAD:AED3DA:B73154:66BF3A6B
Accept-Ranges: bytes
Date: Fri, 16 Aug 2024 11:41:30 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf82